In [67]:
import re
import pandas as pd
import datetime
import mysql.connector


In [80]:
conexao = mysql.connector.connect(host='localhost',user='root',password='',database='producao')
cursor = conexao.cursor()

In [596]:
def LerFilename(revista):
    comando = f'select * from rpis_lidas where rpi="{revista}"'
    cursor.execute(comando)
    resultado = cursor.fetchall()
    #print(len(resultado))
    for row in resultado:
        #print(row[0])
        data_revista = row[1]
    df = pd.DataFrame(resultado)
    data = df[1].astype("string")
    str_data_revista = str(data_revista)
    d=datetime.datetime.strptime(str_data_revista, "%Y-%m-%d")
    #print(d.year)
    aux = str(d.strftime("%d%m%Y"))
    filename = f'revistas/DesenhoIndustrial_{revista}_{aux}.xml'
    return filename, str_data_revista

revista = 2707
filename, data_revista = LerFilename(revista)
print(filename)
print(data_revista)

revistas/DesenhoIndustrial_2707_22112022.xml
2022-11-22


In [597]:
def ConverteNumero(x):
    s = str(x)
    s = s.replace("BR","")
    s = s.replace(" ","")
    pos = s.find('-')
    return s[0:pos]

In [598]:
def ConverteData(data):
    d=datetime.datetime.strptime(data, "%d/%m/%Y")
    return f'{d.year:04}-{d.month:02}-{d.day:02}'

In [599]:
def LerRPI(filename,data_revista):
    file = open(file=filename,mode='r',encoding="utf8")
    file_content_raw=file.read()
    text1=re.compile("<despacho>")
    file_content=text1.split(file_content_raw)
    file.close()
    texto=file_content[0]
    t = re.compile(r'\d{2}/../\d{4}')
    check = t.findall(texto)
    data = ''.join(check)
    data=ConverteData(data)
    if data_revista == data:
        print("Data da RPI "+data_revista+" confere com a data gravada dentro do XML "+data) 
    else:
        print("Data da RPI "+data_revista+" não confere com a data gravada dentro do XML "+data) 
    return file_content

file_content = LerRPI(filename,data_revista)
print("Total registros: "+str(len(file_content)-1))

Data da RPI 2022-11-22 confere com a data gravada dentro do XML 2022-11-22
Total registros: 353


In [600]:
codigo_tag = re.compile('<codigo>(.*?)</codigo>')
titulo_tag = re.compile('<titulo>(.*?)</titulo>')
comentario_tag = re.compile("<comentario inid=\"co\">(.*?)</comentario>") 
nome_completo_tag = re.compile("<nome-completo>(.*?)</nome-completo>") 
data_deposito_tag = re.compile("<data-deposito inid=\"22\">(.*?)</data-deposito>") 

numero_tag = re.compile("<numero inid=\"21\">(.*?)</numero>") 
numero_11_tag = re.compile("<numero inid=\"11\">(.*?)</numero>") 

titulo_pedido_tag = re.compile("<titulo inid=\"54\">(.*?)</titulo>") 
titular_lista_tag = re.compile("<titular-lista>[\s\S]*</titular-lista>")

codigo_list, titulo_list, comentario_list, nome_completo_list, data_deposito_list, numero_list, uf_list, sigla_list, numero_pct_list, data_pct_list, publicacao_internacional_list, numero_ompi_list, data_ompi_list, classe_internacional_list, classe_nacional_list, sigla_prioridade_list, numero_prioridade_list, data_prioridade_list, prioridade_list, prioridade_interna_list, data_fase_nacional_list, titulo_pedido_list, inventor_nome_completo_list, procurador_nome_completo_list, divisao_pedido_list, data_divisao_pedido_list, data_rpi_list, data_concessao_list = ([] for i in range(28))

for line in file_content:
    
    codigo=codigo_tag.findall(line)
    titulo=titulo_tag.findall(line)
    comentario=comentario_tag.findall(line)
    comentario = ', '.join(comentario)

    #to find inventors
    data_deposito=None
    numero=None
    nome_completo=None
    inventor_nome_completo=None
    procurador_nome_completo=None
    titulo_pedido=""
    uf=None
    sigla=None
    classe_nacional=None
    str_classe_nacional = ''
    sigla_prioridade=" "
    numero_prioridade=" "
    data_prioridade=" "
    str_prioridade=''
    data_rpi=" "
    data_concessao=" "
    
    processos=re.findall("<processo-patente>[\s\S]*</processo-patente>",line)
    for processo in processos:
        data_deposito=data_deposito_tag.findall(processo)
        data_deposito=', '.join(data_deposito) # converte lista em string
        numero=numero_tag.findall(processo)
        if len(numero)==0:
            numero=numero_11_tag.findall(processo)

        numero = ', '.join(numero)
        titulo_pedido=titulo_pedido_tag.findall(processo)
        titulo_pedido = ', '.join(titulo_pedido)

        #concessao
        concessoes_lista=re.findall("<concessao inid=\"45\">[\s\S]*</concessao>",processo)
        for concessao_lista in concessoes_lista:
            data_concessao=re.findall("<data>(.*?)</data>",concessao_lista)
            
        #publicacao_nacional (16.1)
        publicacoes_nacional_lista=re.findall("<publicacao-nacional inid=\"44\">[\s\S]*</publicacao-nacional>",processo)
        for publicacao_nacional_lista in publicacoes_nacional_lista:
            data_rpi=re.findall("<data-rpi>(.*?)</data-rpi>",publicacao_nacional_lista)
            
        #titular_lista
        titulares_lista=re.findall("<titular-lista>[\s\S]*</titular-lista>",processo)
        for titular_lista in titulares_lista:
            titulares=re.findall("<titular sequencia=\"1\" inid=\"71\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)
                        
            titulares=re.findall("<titular sequencia=\"1\" inid=\"73\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)

            titulares=re.findall("<titular inid=\"71\" sequencia=\"1\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)
                        
            titulares=re.findall("<titular inid=\"73\" sequencia=\"1\">[\s\S]*</titular>",titular_lista)
            for titular in titulares:
                nome_completo=nome_completo_tag.findall(titular)
                nome_completo = '; '.join(nome_completo)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",titular)
                for endereco in enderecos:
                    uf=re.findall("<uf>(.*?)</uf>",endereco)
                    uf = '; '.join(uf)
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        sigla=re.findall("<sigla>(.*?)</sigla>",pais)
                        sigla = '; '.join(sigla)

        #inventor_lista
        inventores_lista=re.findall("<inventor-lista>[\s\S]*</inventor-lista>",processo)
        for inventor_lista in inventores_lista:
            #print(inventor_lista)
            inventores=re.findall("<inventor sequencia=\"1\" inid=\"72\">[\s\S]*</inventor>",inventor_lista)
            for inventor in inventores:
                inventor_nome_completo=nome_completo_tag.findall(inventor)
                inventor_nome_completo = '; '.join(inventor_nome_completo)

                #inventor_lista
        inventores_lista=re.findall("<inventor-lista>[\s\S]*</inventor-lista>",processo)
        for inventor_lista in inventores_lista:
            #print(inventor_lista)
            inventores=re.findall("<inventor inid=\"72\" sequencia=\"1\">[\s\S]*</inventor>",inventor_lista)
            for inventor in inventores:
                inventor_nome_completo=nome_completo_tag.findall(inventor)
                inventor_nome_completo = '; '.join(inventor_nome_completo)

        #procurador_lista
        procuradores_lista=re.findall("<procurador-lista>[\s\S]*</procurador-lista>",processo)
        for procurador_lista in procuradores_lista:
            procuradores=re.findall("<procurador sequencia=\"1\" inid=\"74\">[\s\S]*</procurador>",procurador_lista)
            for procurador in procuradores:
                procurador_nome_completo=nome_completo_tag.findall(procurador)
                procurador_nome_completo = '; '.join(procurador_nome_completo)

        #procurador_lista
        procuradores_lista=re.findall("<procurador-lista>[\s\S]*</procurador-lista>",processo)
        for procurador_lista in procuradores_lista:
            procuradores=re.findall("<procurador inid=\"74\" sequencia=\"1\">[\s\S]*</procurador>",procurador_lista)
            for procurador in procuradores:
                procurador_nome_completo=nome_completo_tag.findall(procurador)
                procurador_nome_completo = '; '.join(procurador_nome_completo)

        #classificacao_nacional
        classificacoes_nacional_lista=re.findall("<classificacao-nacional-lista>[\s\S]*</classificacao-nacional-lista>",processo)
        for classificacao_nacional_lista in classificacoes_nacional_lista:
            for i in range(1,20):
                #print(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\" >[\s\S]*</classificacao-nacional>")
                classificacoes_nacional=re.findall(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\">[\s\S]*</classificacao-nacional>",classificacao_nacional_lista)
                for classificacao_nacional in classificacoes_nacional:
                    classe_nacional=re.findall(f"<classificacao-nacional inid=\"52\" sequencia=\"{i}\">(.*?)</classificacao-nacional>",classificacao_nacional)
                    if i==1:
                        str_classe_nacional = ', '.join(classe_nacional) # converte lista em string
                    else:
                        str_classe_nacional = str_classe_nacional+'; '+', '.join(classe_nacional) # converte lista em string
                    #print(', '.join(numero)+' '+str_classe)
                    
        #prioridade_unionista
        prioridades_unionista_lista=re.findall("<prioridade-unionista-lista>[\s\S]*</prioridade-unionista-lista>",processo)
        for prioridade_unionista_lista in prioridades_unionista_lista:
            for i in range(1,5):
                prioridades_unionista=re.findall(f"<prioridade-unionista sequencia=\"{i}\" inid=\"30\">[\s\S]*</prioridade-unionista>",prioridade_unionista_lista)
                for prioridade_unionista in prioridades_unionista:
                    #print(prioridade_unionista)
                    sigla_prioridade=re.findall("<sigla-pais inid=\"33\">(.*?)</sigla-pais>",prioridade_unionista)
                    numero_prioridade=re.findall("<numero-prioridade inid=\"32\">(.*?)</numero-prioridade>",prioridade_unionista)
                    data_prioridade=re.findall("<data-prioridade inid=\"31\">(.*?)</data-prioridade>",prioridade_unionista)
                    sigla_prioridade_str=sigla_prioridade[0]
                    numero_prioridade_str=numero_prioridade[0]
                    data_prioridade_str=data_prioridade[0]
                    if i==1:
                        str_prioridade = sigla_prioridade_str+numero_prioridade_str+' '+data_prioridade_str # converte lista em string
                    else:
                        str_prioridade = str_prioridade+'; '+sigla_prioridade_str+numero_prioridade_str+' '+data_prioridade_str

                    numero_str = ', '.join(numero)

 
               
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(codigo)!=0:                             
        codigo_list.append(codigo[0])
        titulo_list.append(titulo[0])
        comentario_list.append(comentario)
        data_deposito_list.append(data_deposito)
        numero_list.append(numero)
        titulo_pedido_list.append(titulo_pedido)
        nome_completo_list.append(nome_completo)
        inventor_nome_completo_list.append(inventor_nome_completo)
        procurador_nome_completo_list.append(procurador_nome_completo)
        uf_list.append(uf)
        sigla_list.append(sigla)
        classe_nacional_list.append(str_classe_nacional)
        prioridade_list.append(str_prioridade)
        sigla_prioridade_list.append(sigla_prioridade[0])
        numero_prioridade_list.append(numero_prioridade[0])
        data_prioridade_list.append(data_prioridade[0])
        data_rpi_list.append(data_rpi[0])
        data_concessao_list.append(data_concessao[0])
        


In [601]:
data_frame = pd.DataFrame(
    {'despacho': codigo_list,
     'numero': numero_list,
     'data_deposito': data_deposito_list,
     'data_publicacao_nacional': data_rpi_list,
     'data_concessao': data_concessao_list,
     'titulo': titulo_pedido_list,
     'depositante': nome_completo_list,
     'uf': uf_list,
     'sigla': sigla_list,
     'inventor': inventor_nome_completo_list,
     'procurador': procurador_nome_completo_list,
     'classe_nacional': classe_nacional_list,
     'prioridade': prioridade_list,
     'sigla_prioridade': sigla_prioridade_list,
     'numero_prioridade': numero_prioridade_list,
     'data_prioridade': data_prioridade_list,
     'comentario': comentario_list,
     'texto_despacho': titulo_list
    })

In [602]:
data_frame["numero_new"] = data_frame["numero"].apply(ConverteNumero)
# https://pt.stackoverflow.com/questions/564421/aplicar-fun%c3%a7%c3%a3o-lamba-para-uma-subtra%c3%a7%c3%a3o-e-soma-python/564425#564425

In [603]:
print(len(data_frame.index))
data_frame[["despacho","numero_new","data_deposito","data_publicacao_nacional","data_concessao","titulo","depositante","uf","sigla","inventor","procurador","classe_nacional","prioridade","sigla_prioridade","numero_prioridade","data_prioridade","comentario","texto_despacho"]]
data_frame.head(1)

353


,despacho,numero,data_deposito,data_publicacao_nacional,data_concessao,titulo,depositante,uf,sigla,inventor,procurador,classe_nacional,prioridade,sigla_prioridade,numero_prioridade,data_prioridade,comentario,texto_despacho,numero_new
0,31,BR 30 2022 006063-1,07/11/2022,,,,YURI YONASHIRO DOS REIS,SP,BR,YURI YONASHIRO DOS REIS,MARCO ANTONIO ROCHA MACHADO,,,,,,Notificação de depósito (Protocolo: 8702201025...,Notificação de Depósito,302022006063


In [604]:
for i, infos in data_frame.iterrows():
    numero = infos.numero_new
    despacho = infos.despacho
    comando = f"INSERT INTO arquivadosdes (id, despacho, numero, data, divisao, anulado, prmexame) VALUES (NULL, '{despacho}', '{numero}', '{data_revista}', '', 0, 0);"
    cursor.execute(comando)
    conexao.commit()

In [605]:
df1 = data_frame.loc[data_frame['despacho']=='31'] 
df2 = data_frame.loc[data_frame['despacho']=='32'] 
df3 = pd.concat([df1, df2], axis=0)
for i, infos in df3.iterrows():
    numero = infos.numero_new
    despacho = infos.despacho
    titular = infos.depositante
    if titular is not None:
        titular = titular.replace("'","")
    inventor = infos.inventor
    if inventor is not None:
        inventor = inventor.replace("''","")
    uf = infos.uf
    pais = infos.sigla
    procurador = infos.procurador
    if procurador is not None:
        procurador = procurador.replace("''","")
    data = data_revista
    data_deposito = ConverteData(infos.data_deposito)
    comando = f"INSERT INTO arquivadosdespub (despacho, numero, titular, inventor, uf, pais, procurador, data, data_deposito, divisao, dataout, despacho_out, data_concessao) VALUES ('{despacho}', '{numero}', '{titular}', '{inventor}', '{uf}', '{pais}', '{procurador}','{data}', '{data_deposito}', '', NULL, '', NULL);"
    #print(comando)
    #break;
    try:
        cursor.execute(comando)
    except (mysql.connector.Error, mysql.connector.Warning) as e:
        print(e)
        print(str(numero))

    conexao.commit()

In [606]:
#confere total de registros gravados
comando = f'select count(*) as total from arquivadosdes where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    if len(data_frame.index) == row[0]:
        print("Sucesso na gravação do total de "+str(row[0])+" registros !")
    else:
        print("Total de registros gravados: "+str(row[0])+" registros, quando toral era de "+len(data_frame.index))



Sucesso na gravação do total de 353 registros !


Processamento encerrado


Processamento encerrado


In [167]:
df1 = data_frame.loc[data_frame['numero_new']=='112021020724']
df1.head(1)
len(df1)
id = df1.index[0]
print(id)
df1.loc[id]
df1.loc[id].despacho

520


'1.3'

In [41]:
df1 = data_frame.loc[data_frame['despacho']=='31'] 
df2 = data_frame.loc[data_frame['despacho']=='32'] 
df3 = pd.concat([df1, df2], axis=0)
df3

,despacho,numero,data_deposito,data_publicacao_nacional,data_concessao,titulo,depositante,uf,sigla,inventor,procurador,classe_nacional,prioridade,sigla_prioridade,numero_prioridade,data_prioridade,comentario,texto_despacho,numero_new
0,31,BR 30 2022 006063-1,07/11/2022,,,,YURI YONASHIRO DOS REIS,SP,BR,YURI YONASHIRO DOS REIS,MARCO ANTONIO ROCHA MACHADO,,,,,,Notificação de depósito (Protocolo: 8702201025...,Notificação de Depósito,302022006063
1,31,BR 30 2022 006065-8,07/11/2022,,,,PAULO ALESSANDRE BIAGE DE SOUZA,SP,BR,PAULO ALESSANDRE BIAGE DE SOUZA,EDUARDA COLLA,,,,,,Notificação de depósito (Protocolo: 8702201026...,Notificação de Depósito,302022006065
2,31,BR 30 2022 006066-6,07/11/2022,,,,PAULO ALESSANDRE BIAGE DE SOUZA,SP,BR,PAULO ALESSANDRE BIAGE DE SOUZA,EDUARDA COLLA,,,,,,Notificação de depósito (Protocolo: 8702201026...,Notificação de Depósito,302022006066
3,31,BR 30 2022 006069-0,07/11/2022,,,,DEERE &amp; COMPANY; DAVID LANE; RYAN KOSIK,,US; CA; CA,WILLIAM K. BORRENPOHL; DAVID LANE; RYAN KOSIK,KASZNAR LEONARDOS PROPRIEDADE INTELECTUAL,,,,,,Notificação de depósito (Protocolo: 8702201027...,Notificação de Depósito,302022006069
4,31,BR 30 2022 006071-2,07/11/2022,,,,FRANCCINO MOVEIS LTDA,MG,BR,DANIELA FERRO GIL,DAIMOND MARCAS E PATENTES LTDA,,,,,,Notificação de depósito (Protocolo: 8702201027...,Notificação de Depósito,302022006071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,31,BR 30 2022 006244-8,11/11/2022,,,,ERICO DE CARVALHO FERREIRA,CE,BR,ERICO DE CARVALHO FERREIRA,WETTOR - BUREAU DE APOIO EMPRESARIAL S/S LTDA. ME,,,,,,Notificação de depósito (Protocolo: 8702201051...,Notificação de Depósito,302022006244
160,31,BR 30 2022 006245-6,11/11/2022,,,,BRENDA IND. E COM. DE MOVEIS LTDA,SC,BR,RODRIGO BUENO,INACIO DAPPER JUNIOR,,,,,,Notificação de depósito (Protocolo: 8702201052...,Notificação de Depósito,302022006245
161,31,BR 32 2022 006067-7,29/04/2022,,,,ARTEFATTO INDUSTRIA E COMERCIO DE ARTEFATOS DE...,RS,BR,CARLOS VARGAS DE MATOS,BRAGA LANES PROPRIEDADE INTELECTUAL E INTERMED...,,,,,,Notificação de depósito (Protocolo: 8702201026...,Notificação de Depósito,322022006067
162,31,BR 32 2022 006068-5,29/04/2022,,,,ARTEFATTO INDUSTRIA E COMERCIO DE ARTEFATOS DE...,RS,BR,CARLOS VARGAS DE MATOS,BRAGA LANES PROPRIEDADE INTELECTUAL E INTERMED...,,,,,,Notificação de depósito (Protocolo: 8702201027...,Notificação de Depósito,322022006068


In [ ]:
CREATE TABLE `arquivadosdespub` (
  `despacho` char(10) NOT NULL,
  `numero` char(15) NOT NULL,
  `titular` text DEFAULT NULL,
  `inventor` text DEFAULT NULL,
  `uf` char(5) DEFAULT NULL,
  `pais` char(5) DEFAULT NULL,
  `procurador` text DEFAULT NULL,
  `data` date DEFAULT NULL,
  `data_deposito` date DEFAULT NULL,
  `divisao` char(15) NOT NULL,
  `dataout` date DEFAULT NULL,
  `despacho_out` char(15) NOT NULL,
  `data_concessao` date DEFAULT NULL
) ENGINE=MyISAM DEFAULT CHARSET=latin1 COMMENT='tabela usada para estatitiscas de residentes, depositos, etc';
ALTER TABLE `arquivadosdespub`
  ADD PRIMARY KEY (`numero`),
  ADD KEY `despacho` (`despacho`);

CREATE TABLE `arquivadosdes` (
  `id` int(11) NOT NULL,
  `despacho` char(30) NOT NULL,
  `numero` char(15) NOT NULL,
  `data` date NOT NULL,
  `divisao` char(11) NOT NULL,
  `anulado` int(11) NOT NULL,
  `prmexame` int(11) NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=latin1 COMMENT='todos os despachos publicados nas RPIs';
ALTER TABLE `arquivadosdes`
  ADD PRIMARY KEY (`id`);